In [1]:
f_text = open('V12.txt', 'r')
f = f_text.readlines()[0]

In [2]:
g_text = open('V12g.txt', 'r')
gamma = g_text.readlines()[0]

In [3]:
f_text.close()
g_text.close()

In [4]:
print(f)

x_{0} x_{3} x_{5} + x_{1} x_{3} x_{5} + x_{2} x_{3} x_{5} + x_{2} x_{5} + x_{3} x_{4} x_{5} + x_{3} x_{5} + x_{3} + x_{4} x_{5} + x_{5} + 1


In [5]:
print(f.replace('_{', '').replace('} x', '*x').replace('}', ''))

x0*x3*x5 + x1*x3*x5 + x2*x3*x5 + x2*x5 + x3*x4*x5 + x3*x5 + x3 + x4*x5 + x5 + 1


In [6]:
def f(x0, x1, x2, x3, x4, x5):
    return (x0*x3*x5 ^^ x1*x3*x5 ^^ x2*x3*x5 ^^ x2*x5 ^^ x3*x4*x5 ^^ x3*x5 ^^ x3 ^^ x4*x5 ^^ x5 ^^ 1)

In [7]:
f(0, 0, 0, 1, 0, 0)

0

In [8]:
f(0, 1, 0, 0, 0, 0)

1

In [9]:
print(gamma[:10])

0100110000


In [10]:
p0 = x^10 + x^3 + 1
p1 = x^10 + x^7 + 1
p2 = x^10 + x^4 + x^3 + x^1 + 1
p3 = x^10 + x^8 + x^3 + x^2 + 1
p4 = x^10 + x^8 + x^4 + x^3 + 1
p5 = x^10 + x^8 + x^5 + x^1 + 1

In [11]:
def to_bits(x):
    x = '{0:06b}'.format(x)
    return [int(bit) for bit in x]

In [12]:
def f_c(x):
    x = to_bits(x)
    return f(x[0], x[1], x[2], x[3], x[4], x[5])

def s_mul(x, y):
    x = to_bits(x)
    y = to_bits(y)
    return sum([x[i]*y[i] for i in range(6)])%2



In [13]:
f_c(1)

0

In [14]:
f_u = [sum((-1)^(f_c(x)^^s_mul(alpha,x)) for x in range (2^6)) for alpha in range (2**6)]

In [15]:
all_non_zero_coef = {i: f_u[i] for i in range(2**6) if f_u[i] != 0}
for a in all_non_zero_coef:
    print('a = ' + str(a) + ', W_f(a) = ' + str(all_non_zero_coef[a]))

a = 4, W_f(a) = -32
a = 5, W_f(a) = -32
a = 10, W_f(a) = 16
a = 11, W_f(a) = -16
a = 14, W_f(a) = 16
a = 15, W_f(a) = -16
a = 48, W_f(a) = 16
a = 49, W_f(a) = -16
a = 52, W_f(a) = -16
a = 53, W_f(a) = 16


In [16]:
j = 1
for a in all_non_zero_coef:
    c = 0 
    one = 0
    if all_non_zero_coef[a] < 0:
        one = 1
    bit_a = to_bits(a)
    g = ''.join("x" + str(i) + " ⊕ "  for i in range(6) if bit_a[i] == 1)[:-2] + ("⊕ 1" if one == 1 else '')
    P = (1 + (-1)**c*all_non_zero_coef[a]/2**6)/2
    print("g = " +  g + ", P(f(x) = g(x))  =" + str(P))
    #print ("Pr{f(x) = g(x)} =" + str((1 + (-1)**c*all_non_zero_coef[a]/2**6)/2))
    j+= 1

g = x3 ⊕ 1, P(f(x) = g(x))  =1/4
g = x3 ⊕ x5 ⊕ 1, P(f(x) = g(x))  =1/4
g = x2 ⊕ x4 , P(f(x) = g(x))  =5/8
g = x2 ⊕ x4 ⊕ x5 ⊕ 1, P(f(x) = g(x))  =3/8
g = x2 ⊕ x3 ⊕ x4 , P(f(x) = g(x))  =5/8
g = x2 ⊕ x3 ⊕ x4 ⊕ x5 ⊕ 1, P(f(x) = g(x))  =3/8
g = x0 ⊕ x1 , P(f(x) = g(x))  =5/8
g = x0 ⊕ x1 ⊕ x5 ⊕ 1, P(f(x) = g(x))  =3/8
g = x0 ⊕ x1 ⊕ x3 ⊕ 1, P(f(x) = g(x))  =3/8
g = x0 ⊕ x1 ⊕ x3 ⊕ x5 , P(f(x) = g(x))  =5/8


In [17]:
%%cython

from libc.stdlib cimport malloc, free

cdef int LENGTH = 10

def gen_sequence(int poly, int init_state, int nbits):
    cdef int curr_state = init_state, s, i, j
    cdef int *res_array = <int *>malloc(sizeof(int)*nbits)
    try:
        for i in range(nbits):
            res_array[i] = (curr_state >> (LENGTH - 1))&1
            s = 0
            for j in range(LENGTH):
                s = s^((( curr_state & poly ) >> j)&1)
            curr_state = (curr_state << 1)|s
        return [x for x in res_array[: nbits]]
    finally:
        free(res_array)

In [18]:
delta = 0.01

In [20]:
e = 0.5
m = 10
k = 1
T = int(8*(e^(-2))*math.log(2^(m*k)*delta^(-1)))
k = 2
T2 = int(8*(e^(-2))*math.log(2^(m*k)*delta^(-1)))

In [21]:
print(T)
print(T2)

369
590


In [22]:
def distance(x, y):
    return sum([abs((x[i]) - (y[i])) for i in range(len(x))])

In [23]:
p = [p0, p1, p2, p3, p4, p5]
bin_poly = [''.join([str(i) for i in poly.list()])[:-1] for poly in p]
int_poly = [int(integer, 2) for integer in bin_poly]

In [24]:
int_poly

[576, 516, 864, 706, 610, 786]

In [25]:
gamma = [int(g) for g in gamma]

In [26]:
def g3(x0, x1, x2, x3, x4, x5):
    return x2 ^^ x4 ^^ 1

def g1(x0, x1, x2, x3, x4, x5):
    return x3 ^^ 1

def g5(x0, x1, x2, x3, x4, x5):
    return x2 ^^ x3 ^^ x4 ^^ 1

def g9(x0, x1, x2, x3, x4, x5):
    return x0 ^^ x1 ^^ x3 ^^ x5 ^^ 1

def g2(x0, x1, x2, x3, x4, x5):
    return x3 ^^ x5 ^^ 1

In [27]:
g2 = lambda x3, x5: x3 ^^ x5 ^^ 1
g3 = lambda x2, x4: x2 ^^ x4 ^^ 1
g1 = lambda x3: x3^^1
g5 = lambda x2, x3, x4 : x2 ^^ x3 ^^ x4 ^^ 1
g9 = lambda x0, x1, x3, x5: x0 ^^ x1 ^^ x3 ^^ x5 ^^ 1
g7 = lambda x0, x1: x0 ^^ x1 ^^ 1
T_range = range(T)

In [ ]:
%%time
states = {}
for i in range(2**10):
    x0 = gen_sequence(int_poly[0], i, T)
    for j in range(2**10):
        x1 = gen_sequence(int_poly[1], i, T)
        sequence = [g7(x0[j], x1[j]) for j in T_range]
    states[i] = distance(sequence, gamma[:T])
dist = min(states.values())
print('min distance = ', dist)
states = [i for i in states if states[i] == dist]
for x0, x1 in states:
    print(x0, x1) 

In [35]:
%%time
states = {}
for i in range(2**10):
    x3 = gen_sequence(int_poly[3], i, int(T))
    sequence = [g1(x3[j]) for j in T_range]
    states[i] = distance(sequence, gamma[:T])
dist = min(states.values())
print('min distance = ', dist)
states = [i for i in states if states[i] == dist]
for x3 in states:
    print(x3)
    x_3 = x3

#x3 = gen_sequence(int_poly[3], 381, T)

min distance =  81
751
CPU times: user 53.9 ms, sys: 1.07 s, total: 1.13 s
Wall time: 1.2 s


In [28]:
x3 = gen_sequence(int_poly[3], x_3, T)

g1 = x3 xor 1

In [31]:
%%time
states = {}
for i in range(2**10):
    x5 = gen_sequence(int_poly[5], i, int(T))
    sequence = [g2(0, 0, 0, x3[j], 0, x5[j]) for j in range(T)]
    states[i] = distance(sequence, gamma[:T])
dist = min(states.values())
print('min distance = ', dist)
states = [i for i in states if states[i] == dist]
for x__5 in states:
    print(x__5)
    x_5 = x__5


TypeError: <lambda>() takes 2 positional arguments but 6 were given

In [32]:
%%time
states = {}
for i in range(2**10):
    x5 = gen_sequence(int_poly[5], i, int(T))
    sequence = [g2(x3[j], x5[j]) for j in range(T)]
    states[i] = distance(sequence, gamma[:T])
dist = min(states.values())
print('min distance = ', dist)
states = [i for i in states if states[i] == dist]
for x__5 in states:
    print(x__5)
    x_5 = x__5


min distance =  157
639
683
934
CPU times: user 795 ms, sys: 550 ms, total: 1.34 s
Wall time: 1.51 s


In [ ]:
%%time
x3 = gen_sequence(int_poly[3], 381, T)
states = {}
for i in range(2**10):
    x2 = gen_sequence(int_poly[2], i, T)
    for j in range(2**10):
        x4 = gen_sequence(int_poly[4], i, T)
        sequence = [g5(0, 0, x2[j], x3[j], x4[j], 0) for j in range(T)]
    states[i] = distance(sequence, gamma[:T])
dist = min(states.values())
print('min distance = ', dist)
states = [i for i in states if states[i] == dist]
for x2, x4 in states:
    print(x2," ", x4) 

In [ ]:
%%time
states = {}
for i in range(2**10):
    sequence = gen_sequence(int_poly[3], i, int(T))
    #sequence = g5(int_poly[1], i, int(T))
    states[i] = distance(sequence, gamma[:T])
dist = min(states.values())
print('min distance = ', dist)
states = [i for i in states if states[i] == dist]
for x5 in states:
    print(x5) 